In [203]:
%config IPCompleter.greedy=True
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [204]:
import os, sys, re, math, datetime as dt, pandas as pd, numpy as np, time
import logging
import matplotlib.pyplot as plt
from string import Template
from IPython.display import display, HTML

logging.basicConfig(format='%(asctime)s [%(name)s:%(lineno)d:%(funcName)s] [%(levelname)s] %(message)s', level=logging.INFO)

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_colwidth', 5000)
pd.set_option('display.width', 5000)

def display_df(df):
    display(df.head(4))
    print(df.shape)

In [205]:
import fastf1 as f1

session = f1.get_session(2023, 9, 'R')
cache_dir = f"/Users/hannahwang/Projects/formula_1/src/fastf1_cache"
f1.Cache.enable_cache(cache_dir=cache_dir)

# schedule = f1.get_event_schedule(2023)
# # print(session.event)
# display_df(schedule)
# schedule.dtypes
session

2023 Season Round 9: Austrian Grand Prix - Race

In [206]:
session.load()

core           INFO 	Loading data for Austrian Grand Prix - Race [v3.1.6]
2024-01-04 17:11:19,456 [fastf1.fastf1.core:1248:load] [INFO] Loading data for Austrian Grand Prix - Race [v3.1.6]
req            INFO 	Using cached data for session_info
2024-01-04 17:11:19,458 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for session_info
req            INFO 	Using cached data for driver_info
2024-01-04 17:11:19,460 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
2024-01-04 17:11:20,432 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
2024-01-04 17:11:20,434 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
2024-01-04 17:11:20,435 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using

In [207]:
driver_id = "VER"
laps = session.laps.pick_driver(driver_id)

In [208]:
laps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 01:03:05.115000,VER,1,0 days 00:01:17.639000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:31.613000,0 days 00:00:25.440000,NaT,0 days 01:02:39.687000,0 days 01:03:05.187000,292.0,233.0,109.0,297.0,False,MEDIUM,1.0,True,Red Bull Racing,0 days 01:01:47.219000,2023-07-02 13:02:48.211,24,1.0,False,,False,False
1,0 days 01:05:00.594000,VER,1,0 days 00:01:55.479000,2.0,1.0,NaT,0 days 01:04:57.200000,0 days 00:00:31.698000,0 days 00:00:46.293000,0 days 00:00:37.488000,0 days 01:03:36.813000,0 days 01:04:23.106000,0 days 01:05:00.594000,176.0,184.0,NaN,195.0,False,MEDIUM,2.0,True,Red Bull Racing,0 days 01:03:05.115000,2023-07-02 13:04:06.107,4,1.0,False,,False,False
2,0 days 01:07:05.315000,VER,1,0 days 00:02:04.721000,3.0,2.0,0 days 01:05:13.560000,NaT,0 days 00:00:37.877000,0 days 00:00:46.298000,0 days 00:00:40.546000,0 days 01:05:38.471000,0 days 01:06:24.769000,0 days 01:07:05.315000,191.0,61.0,270.0,236.0,False,MEDIUM,3.0,False,Red Bull Racing,0 days 01:05:00.594000,2023-07-02 13:06:01.586,4,1.0,False,,False,False
3,0 days 01:08:15.006000,VER,1,0 days 00:01:09.691000,4.0,2.0,NaT,NaT,0 days 00:00:17.618000,0 days 00:00:30.970000,0 days 00:00:21.103000,0 days 01:07:22.933000,0 days 01:07:53.903000,0 days 01:08:15.006000,293.0,230.0,270.0,297.0,True,MEDIUM,4.0,False,Red Bull Racing,0 days 01:07:05.315000,2023-07-02 13:08:06.307,1,1.0,False,,False,False
4,0 days 01:09:25.032000,VER,1,0 days 00:01:10.026000,5.0,2.0,NaT,NaT,0 days 00:00:17.716000,0 days 00:00:31.158000,0 days 00:00:21.152000,0 days 01:08:32.722000,0 days 01:09:03.880000,0 days 01:09:25.032000,285.0,230.0,270.0,289.0,False,MEDIUM,5.0,False,Red Bull Racing,0 days 01:08:15.006000,2023-07-02 13:09:15.998,1,1.0,False,,False,True


In [209]:
colors = """aggrnyl     agsunset    blackbody   bluered     blues       blugrn      bluyl       brwnyl
bugn        bupu        burg        burgyl      cividis     darkmint    electric    emrld
gnbu        greens      greys       hot         inferno     jet         magenta     magma
mint        orrd        oranges     oryel       peach       pinkyl      plasma      plotly3
pubu        pubugn      purd        purp        purples     purpor      rainbow     rdbu
rdpu        redor       reds        sunset      sunsetdark  teal        tealgrn     turbo
viridis     ylgn        ylgnbu      ylorbr      ylorrd      algae       amp         deep
dense       gray        haline      ice         matter      solar       speed       tempo
thermal     turbid      armyrose    brbg        earth       fall        geyser      prgn
piyg        picnic      portland    puor        rdgy        rdylbu      rdylgn      spectral
tealrose    temps       tropic      balance     curl        delta       oxy         edge
hsv         icefire     phase       twilight    mrybm       mygbm"""

colors = colors.split()
colors

px.colors.sequential.swatches

<function _plotly_utils.colors.sequential.swatches(template=None)>

In [210]:
import plotly.graph_objects as go
import plotly.express as px
from datetime import timedelta
from plotly.subplots import make_subplots
import datetime

from plotly.express.colors import sample_colorscale
from sklearn.preprocessing import minmax_scale

lap_numbers = [50,2,25,46,27,48]

lap_numbers.sort()

# colors = px.colors.sequential.Blackbody

colors_ = [i for i in range(0,int(len(lap_numbers)*1.3))] 
discrete_colors = sample_colorscale('Reds', minmax_scale(colors_))

discrete_colors.reverse()


fig = make_subplots(rows=5, cols=1,
                   shared_xaxes=True,
                    vertical_spacing=0.05
                   )

print(discrete_colors)

for index, lap_n in enumerate(lap_numbers):
    print(lap_n)
    lap_tel = laps.pick_laps([lap_n]).get_telemetry()
    # display(lap_tel.head())
    date = lap_tel.Date.iloc[0]
    

    # lap_tel['millis'] = lap_tel.Time.map(lambda x: int(x/timedelta(milliseconds=1)))
    # lap_tel['str_time'] = lap_tel.millis.map(lambda x: f"{x/1000/60}:{x/1000}")
    lap_tel['time'] = lap_tel.Time.map(lambda x: x + datetime.datetime(date.year,date.month,date.day,0,0,0,0))
    
    # print(lap_tel.dtypes)
    color = discrete_colors[index]
    # rgba = my_color = ('rgba('+str(int(color[5:], 16))+','+
    #             str(int(color[3:5], 16))+','+
    #             str(int(color[1:3], 16)))
    # print(rgba)
    
    line_config = dict(color=f"{color[:-1]},0.8)", width=1.5,)

    time_template = '<br>%{x|%M:%S.%L}<br>'
    distance_template = '<br>%{x:d} m<br>'
    
    
    x_data = lap_tel.time
    x_template = time_template
    
    fig_speed = go.Scatter(x=x_data, y=lap_tel.Speed,
                           mode='lines',
                           name=f'lap {lap_n}',
                           opacity=1,
                           line=line_config,
                           hovertemplate = x_template + '%{y} km/h',
                  )
    fig_ngear = go.Scatter(x=x_data, y=lap_tel.nGear,
                           mode='lines',
                           name=f'lap {lap_n}',
                           opacity=1,
                           line=line_config,
                           hovertemplate = x_template + 'gear %{y}',
                  )
    fig_throttle = go.Scatter(x=x_data, y=lap_tel.Throttle,
                           mode='lines',
                           name=f'lap {lap_n}',
                           opacity=1,
                           line=line_config,
                           hovertemplate = x_template + 'throttle: %{y}%',
                  )
    fig_rpm = go.Scatter(x=x_data, y=lap_tel.RPM,
                           mode='lines',
                           name=f'lap {lap_n}',
                           opacity=1,
                           line=line_config,
                           hovertemplate = x_template + '%{y} RPM',
                  )
    fig_brake = go.Scatter(x=x_data, y=lap_tel.Brake,
                           mode='lines',
                           name=f'lap {lap_n}',
                           opacity=1,
                           line=line_config,
                           hovertemplate = x_template + 'braking: %{y}',
                  )
    # fig_drs
    # fig_status

    fig.add_trace(fig_speed, row=1, col=1)
    fig.add_trace(fig_ngear, row=2, col=1)
    fig.add_trace(fig_throttle, row=3, col=1)
    fig.add_trace(fig_rpm, row=4, col=1)
    fig.add_trace(fig_brake, row=5, col=1)
    
fig['layout']['xaxis']['title']='Time [ms]'

fig['layout']['yaxis']['title']='Speed [km/h]'
fig['layout']['yaxis2']['title']='Gear'
fig['layout']['yaxis3']['title']='Throttle %'
fig['layout']['yaxis4']['title']='RPM'
fig['layout']['yaxis5']['title']='Brake'

# fig.add_vline(x=15_000, line_width=3, line_color="green")
fig.update_traces(xaxis="x2")

x_axis_config = dict(
                    showline=True,
                     showgrid=True,
                     showticklabels=True,
                     # linecolor='rgb(0,0,0)',
                     # linewidth=5,
                     mirror=True,
                     showspikes = True,
                     spikemode  = 'toaxis+across',
                     spikesnap = 'cursor',
                    )

fig['layout']['xaxis'] = x_axis_config
# fig['layout']['xaxis2'] = x_axis_config
# fig['layout']['xaxis3'] = x_axis_config
# fig['layout']['xaxis4'] = x_axis_config
# fig['layout']['xaxis5'] = x_axis_config

fig.update_layout(height=1000, width=1200, title_text="Statistics", showlegend=False,
                  hovermode='x unified',
                  hoverlabel=dict(
                      bgcolor="rgba(25,25,25,0.9)",
                      font_size=10,
                      font_color='white'
                      # font_family="Rockwell"
                  ),
                  plot_bgcolor='#252525'
                 )


fig.show()


['rgb(103, 0, 13)', 'rgb(178, 18, 24)', 'rgb(227, 47, 39)', 'rgb(251, 106, 74)', 'rgb(252, 160, 130)', 'rgb(253, 212, 194)', 'rgb(255, 245, 240)']
2
25
27
46
48
50


In [224]:
session.laps.pick_driver("HUL").head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
847,0 days 01:03:17.170000,HUL,27,0 days 00:01:29.694000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:32.369000,0 days 00:00:34.445000,NaT,0 days 01:02:42.812000,0 days 01:03:17.280000,299.0,227.0,102.0,298.0,False,MEDIUM,4.0,False,Haas F1 Team,0 days 01:01:47.219000,2023-07-02 13:02:48.211,24,7.0,False,,False,False
848,0 days 01:05:12.776000,HUL,27,0 days 00:01:55.606000,2.0,1.0,NaT,0 days 01:05:08.997000,0 days 00:00:31.064000,0 days 00:00:41.803000,0 days 00:00:42.739000,0 days 01:03:48.289000,0 days 01:04:30.092000,0 days 01:05:12.831000,236.0,131.0,NaN,266.0,False,MEDIUM,5.0,False,Haas F1 Team,0 days 01:03:17.170000,2023-07-02 13:04:18.162,4,7.0,False,,False,False
849,0 days 01:07:08.058000,HUL,27,0 days 00:01:55.282000,3.0,2.0,0 days 01:05:25.684000,NaT,0 days 00:00:33.612000,0 days 00:00:43.882000,0 days 00:00:37.788000,0 days 01:05:46.443000,0 days 01:06:30.325000,0 days 01:07:08.113000,265.0,67.0,271.0,262.0,False,MEDIUM,6.0,False,Haas F1 Team,0 days 01:05:12.776000,2023-07-02 13:06:13.768,4,7.0,False,,False,False
850,0 days 01:08:20.119000,HUL,27,0 days 00:01:12.061000,4.0,2.0,NaT,NaT,0 days 00:00:17.983000,0 days 00:00:32.180000,0 days 00:00:21.898000,0 days 01:07:26.096000,0 days 01:07:58.276000,0 days 01:08:20.174000,295.0,223.0,271.0,296.0,True,MEDIUM,7.0,False,Haas F1 Team,0 days 01:07:08.058000,2023-07-02 13:08:09.050,1,7.0,False,,False,False
851,0 days 01:09:31.735000,HUL,27,0 days 00:01:11.616000,5.0,2.0,NaT,NaT,0 days 00:00:17.692000,0 days 00:00:32.078000,0 days 00:00:21.846000,0 days 01:08:37.866000,0 days 01:09:09.944000,0 days 01:09:31.790000,294.0,225.0,270.0,298.0,True,MEDIUM,8.0,False,Haas F1 Team,0 days 01:08:20.119000,2023-07-02 13:09:21.111,1,7.0,False,,False,True


In [226]:
driver_pos = session.laps.pick_driver("HUL").Position
laps_participated = session.laps.pick_driver("HUL").LapNumber
laps_participated = laps_participated[driver_pos.isna()==False]
laps_participated.tolist()

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]